<a href="https://colab.research.google.com/github/SkyRanger2010/DE2024_PY/blob/main/HomeWork_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
temperature_file_name = '/content/drive/MyDrive/Colab Notebooks/hw/city_weather.csv'
loc_city_name = '/content/drive/MyDrive/Colab Notebooks/hw/district_to_city.csv'
city_mapper_name = '/content/drive/MyDrive/Colab Notebooks/hw/city_mapper.csv'
district_mapper_name = '/content/drive/MyDrive/Colab Notebooks/hw/district_mapper.csv'

In [ ]:
import csv
import pandas as pd

df=pd.read_csv(loc_city_name)

def map_city(line:list)->list:  #mapper для городов
    city = line[0]
    day_temp = float(line[2])
    night_temp = float(line[3])
    mid_temp = round((day_temp+night_temp)/2,1)
    district = df[df['city']==city].federal_district.values[0]
    count_index = 1
    return [city, district, mid_temp,count_index]

def map_district(line:list)->list: #mapper для округов
    city = line[0]
    day_temp = float(line[2])
    night_temp = float(line[3])
    mid_temp = round((day_temp+night_temp)/2,1)
    district = df[df['city']==city].federal_district.values[0]
    count_index = 1
    return [district, mid_temp,count_index]

with open(temperature_file_name, "r") as temp_file:
  with open(city_mapper_name, "w") as city_mapper:
    with open(district_mapper_name, "w") as district_mapper:

      temp_file_reader = csv.reader(temp_file)
      city_mapper_writer = csv.writer(city_mapper)
      city_mapper_writer.writerow(['city','district','mean_temp','count_index'])
      district_mapper_writer = csv.writer(district_mapper)
      district_mapper_writer.writerow(['district','mean_temp','count_index'])

      for i, line in enumerate(temp_file_reader):
          if not i: continue
          city_mapper_writer.writerow(map_city(line))
          district_mapper_writer.writerow( map_district(line))




In [ ]:
!cat "$city_mapper_name" | sort > cities_sorted.csv

In [ ]:
!cat "$district_mapper_name" | sort > districs_sorted.csv

In [ ]:
district_name = None
city_name = None
cnt = 0
# reducer для округов

with open('district_reduce.csv', "w") as res:
  res_writer = csv.writer(res)
  res_writer.writerow(['district','mean_temperature'])
  with open("districs_sorted.csv", "r") as f:
    districs_sorted_reader = csv.reader(f)
    next(districs_sorted_reader,None)
    for line in districs_sorted_reader:
        district, mean_temp, count = line
        if district_name is None :
            district_name = district
            sum_mean_temp = float(mean_temp)
            cnt = int(count)
        else:
            if district_name == district:
                sum_mean_temp += float(mean_temp)
                cnt += int(count)
            else:
                res_mean_temp = round(sum_mean_temp/cnt,2)
                res_writer.writerow([district_name, str(res_mean_temp)])
                district_name = district
                sum_mean_temp = float(mean_temp)
                cnt = int(count)
  res_mean_temp = round(sum_mean_temp/cnt,2)
  res_writer.writerow([district_name, str(res_mean_temp)])

df = pd.read_csv('/content/district_reduce.csv')

# reducer для городов

with open('cities_reduce.csv', "w") as res:
  res_writer = csv.writer(res)
  res_writer.writerow(['city','federal_district','district_mean_temp','city_mean_temperature', 'district_temp_diff'])
  with open("cities_sorted.csv", "r") as f:
    cities_sorted_reader = csv.reader(f)
    next(cities_sorted_reader,None)
    for line in cities_sorted_reader:
        city, district, mean_temp, count = line
        if city_name is None :
            city_name = city
            sum_mean_temp = float(mean_temp)
            district_name = district
            cnt = int(count)
        else:
            if city_name == city:
                sum_mean_temp += float(mean_temp)
                cnt += int(count)
            else:
                res_mean_temp = round(sum_mean_temp/cnt,2)
                district_mean_temp=df[df['district']==district_name]['mean_temperature'].values[0]
                disrict_temp_diff = round(res_mean_temp - district_mean_temp,2)
                res_writer.writerow([city_name, district_name, str(district_mean_temp), str(res_mean_temp), str(disrict_temp_diff)])
                city_name = city
                district_name = district
                sum_mean_temp = float(mean_temp)
                cnt = int(count)
  res_mean_temp = round(sum_mean_temp/cnt,2)
  district_mean_temp=df[df['district']==district_name]['mean_temperature'].values[0]
  disrict_temp_diff = round(res_mean_temp - district_mean_temp,2)
  res_writer.writerow([city_name, district_name, str(district_mean_temp), str(res_mean_temp), str(disrict_temp_diff)])



In [ ]:
cities = pd.read_csv('/content/cities_reduce.csv')
cities

,city,federal_district,district_mean_temp,city_mean_temperature,district_temp_diff
0,Абакан,Сибирский ФО,-4.26,-1.65,2.61
1,Анадырь,Дальневосточный ФО,-6.37,-12.63,-6.26
2,Архангельск,Северо-Западный ФО,-0.82,-2.75,-1.93
3,Астрахань,Южный ФО,6.86,7.82,0.96
4,Барнаул,Сибирский ФО,-4.26,-3.82,0.44
...,...,...,...,...,...
67,Челябинск,Уральский ФО,-0.47,0.83,1.30
68,Чита,Сибирский ФО,-4.26,-6.01,-1.75
69,Элиста,Южный ФО,6.86,7.19,0.33
70,Южно-Сахалинск,Дальневосточный ФО,-6.37,-3.61,2.76
